In [1]:
from itertools import islice

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np

from openpyxl import load_workbook

DATA_FILE = "задание_dsc339a3a1-1431-4382-b898-9b0a9eef77e2.xlsx"

In [2]:
def load_data(sheetnum,
              fname=DATA_FILE, 
              col_names=("name", "gender", "document", "country"),
             ):
    wb = load_workbook(filename=DATA_FILE)
    ws = wb.worksheets[sheetnum]

    data = pd.DataFrame(islice(ws.values,1, None))
    data.columns = col_names
    return data

def split_name(df):
    data = df.copy()
    names = data["name"].str.split("\s+", expand=True)
    col_names = [f"name{i}" for i in names.columns]
    names.columns = col_names
    data.loc[:, col_names] = names[col_names]
    return data



def match_cosine(fact_data, test_data):
    matching_vec = fact_data["name"].unique()
    tfidf_vectorizer = TfidfVectorizer()
    sparse_matrix = tfidf_vectorizer.fit_transform(matching_vec)
    term_matrix = sparse_matrix.toarray()
    
    def _(to_match):
        match_trf = tfidf_vectorizer.transform([to_match]).toarray()
        match_cosine = cosine_similarity(term_matrix, match_trf)
        return (to_match, matching_vec[np.argmax(match_cosine)], match_cosine)
    
    return [_(name) for name in test_data["name"]]

def match_gender(fact_data, test_data):
    columns = [c for c in fact_data.columns if c[:len("name")] == "name" and len(c)>len("name")]
    keys = ["name", "gender"] + columns
    matched = pd.DataFrame(dict(zip(keys, [[] for i in range(len(keys))])))
    for i,c in enumerate(columns):
        a = fact_data[[c, "gender"]].drop_duplicates().dropna()
        b = a.groupby(c).count().reset_index()
        twin_list = b[b["gender"]>1][c].tolist()
        lookup = a[~a[c].isin(twin_list)]
        idx = test_data[c].isin(a[c]) & \
                          (~test_data[c].isin(twin_list))
        if len(matched.index) > 0:
            for j in range(i):
                idx = idx & (~test_data[f"name{j}"].isin(matched[f"name{j}"]))
        combo = test_data[idx][["name"] + columns].drop_duplicates()
        res = pd.merge(combo, lookup, on=c)
        matched = pd.concat([matched, res])
    return matched

 Загрузим данные и разделим имена на части.

In [4]:
fact = load_data(1)
test = load_data(2)
fact_data = split_name(fact)
test_data = split_name(test)

## Присвоение признака пола

Сколько уникальных фамилий в "фактическом" датасете:

In [5]:
fact_data["name0"].unique().shape[0]

5895

Сколько фамилий в "фактическом" датасете принадлежат людям обоих полов:

In [6]:
a = fact_data[["name0", "gender"]].drop_duplicates()
a = a.groupby("name0").count().reset_index()
twin_gender = a[a["gender"]>1]
len(twin_gender.index)


218

Присвоить признак пола по имени:

In [7]:
matched_gender = match_gender(fact_data, test_data)

У части тестовых записей отсутствуют аналоги в фактических данных

In [8]:
a = test_data[["name", "name0", "name1", "name2", "name3", "name4", "name5"]].drop_duplicates()
missing = a[~a["name"].isin(matched_gender["name"])]
missing

,name,name0,name1,name2,name3,name4,name5
12,АБДАЗИЗОВА ОМУРКАН,АБДАЗИЗОВА,ОМУРКАН,None,None,None,None
17,АБДИЛАМИТОВ МУМИН,АБДИЛАМИТОВ,МУМИН,None,None,None,None
20,АБДИНАЗИМ КЫЗЫ МУНАИМ,АБДИНАЗИМ,КЫЗЫ,МУНАИМ,None,None,None
26,АБДУГАНИЕВА МАЛОХАТ МУФТОХИДДИН КИЗИ,АБДУГАНИЕВА,МАЛОХАТ,МУФТОХИДДИН,КИЗИ,None,None
43,АБДУМАНАБОВ МУРАДЖОН СОВОНКУЛОВИЧ,АБДУМАНАБОВ,МУРАДЖОН,СОВОНКУЛОВИЧ,None,None,None
...,...,...,...,...,...,...,...
2431,ЩАННИКОВА НАТАЛЬЯ ЮРЬЕВНА,ЩАННИКОВА,НАТАЛЬЯ,ЮРЬЕВНА,None,None,None
2460,ЭРНАЗАРОВ САФАРАЛИ ГАЙИПОВИЧ,ЭРНАЗАРОВ,САФАРАЛИ,ГАЙИПОВИЧ,None,None,None
2461,ЭРНАФАСОВ ЭШМУРОД КУРОКБОЙ УГЛИ,ЭРНАФАСОВ,ЭШМУРОД,КУРОКБОЙ,УГЛИ,None,None
2464,ЭСЕНБАЕВА ГУЛСИНАЙ,ЭСЕНБАЕВА,ГУЛСИНАЙ,None,None,None,None


Попробуем найти близкие аналоги по косинусному расстоянию:

In [10]:
a = list(zip(*match_cosine(fact_data, missing)))
cos_matched = pd.DataFrame({"test_name" : a[0], "fact_name" : a[1]})
b = fact_data.loc[fact_data["name"].isin(a[1]), ["name", "gender"]].drop_duplicates()
b = b.rename(columns={"name" : "fact_name"})
res = pd.merge(cos_matched, b, on="fact_name").drop(columns="fact_name")
res = res.rename(columns={"test_name" : "name"})

In [12]:
full_match = pd.concat([matched_gender[["name", "gender"]], res])

Проверим результат на базовую вменяемость и обнаружим наблюдения, которые явно получили неправильный пол. Поскольку таких случаев не слишком много, мы можем проверить их "на глаз" и если исключений не обнаружим, то заменим пол на противоположный для всех выявленных записей.

In [13]:
idx = (full_match["name"].str.contains(r"ОВНА\b") & (full_match["gender"] == "М")) | \
    (full_match["name"].str.contains(r"ОВИЧ\b") & (full_match["gender"] == "Ж"))
full_match[idx]

,name,gender
44,АДАМЯН РИТА ЗАВЕНОВНА,М
101,АМОЯН АРАМ САРГИСОВИЧ,Ж
124,АТАБЕКЯН ДЖИВАН НВЕРОВИЧ,Ж
174,БАКОЯН АНТАРАМ ГАГИКОВНА,М
193,БИЛЫК РОМАН ОДИЛОВИЧ,Ж
203,БОНДАРЬ СВЕТЛАНА ПОЛИКАРПОВНА,М
233,ГАРИБЯН АРЦВИК РАФИКОВНА,М
259,ГЕВОНДЯН АСТГИК АМАЯКОВНА,М
325,ДЬЯЧЕНКО ВАЛЕРИЯ АЛЕКСАНДРОВНА,М
340,ЖЕНИШБЕК КЫЗЫ МИЛАННА АЛЕКСАНДРОВНА,М


"ИВАНОВИЧ САМИРА ВИКТОРОВНА" очевидно определена правильно как женщина, что наводит нас на мысль о целесообразности дополнительного теста: 

In [14]:
idx2 = (full_match["name"].str.contains(r"ОВНА\b") & full_match["name"].str.contains(r"ОВИЧ\b"))
full_match[idx2]

,name,gender
896,ИВАНОВИЧ САМИРА ВИКТОРОВНА,Ж


Похоже, что Самира Викторовна уникальна, поэтому ее мы исключаем, а для остальных заменяем пол на противоположный:

In [15]:
exceptions = full_match[idx].copy()
exceptions = exceptions.drop(exceptions[exceptions["name"]=="ИВАНОВИЧ САМИРА ВИКТОРОВНА"].index)
gen_flip = {"Ж" : "М", "М" : "Ж"}
exceptions.loc[:, "gender"] = [gen_flip[gen] for gen in exceptions["gender"]]

full_match.loc[full_match["name"].isin(exceptions["name"]), "gender"] = exceptions["gender"]

In [16]:
full_match

,name,gender
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,М
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,Ж
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,М
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,М
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,М
...,...,...
242,ШАРИПОВ АБДИМАЛИК АЛИЖАНОВИЧ,М
243,ШИРАЛИЕВА СААДЕТ КАМИЛ КЫЗЫ,Ж
244,ЩАННИКОВА НАТАЛЬЯ ЮРЬЕВНА,Ж
245,ЭРНАЗАРОВ САФАРАЛИ ГАЙИПОВИЧ,М


## Присвоение признака страны

Наличие уникальных схем номеров документа для стран в фактических данных позволяет нам присвоить страну некоторой части наблюдений в тестовых данных. Эти уникальные схемы: 

In [17]:
a = fact_data[["document", "country"]].drop_duplicates()
b = a.groupby(["document"]).count().reset_index()
unq_pat = b[b["country"]==1]
unq_doc_lookup = fact_data.loc[fact_data["document"].isin(unq_pat["document"]), ["document", "country"]].drop_duplicates()
unq_doc_lookup

,document,country
0,AP!!!!!!,АРМЕНИЯ
1,AB!!!!!!!,УЗБЕКИСТАН
2,СР!!!!!!!,УЗБЕКИСТАН
11,AZII!!!!!!,АЗЕРБАЙДЖАН
14,C!!!!!!!!,АЗЕРБАЙДЖАН
...,...,...
9281,PKGZAC!!!!!!,КИРГИЗИЯ
9341,CH!!!!!!!,УЗБЕКИСТАН
9397,!!!!!!!СО,АЗЕРБАЙДЖАН
9471,PUZBCB!!!!!!!,УЗБЕКИСТАН


Присвоим национальность части тестовых наблюдений:

In [18]:
a = test_data.copy()
a = a.drop(columns="country")
doc_matched = pd.merge(a, unq_doc_lookup)
doc_matched

,name,gender,document,name0,name1,name2,name3,name4,name5,country
0,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,None,P!!!!!!!!,АББАСОВ,ЭЛЬФАГ,ИБРАГИМ,ОГЛЫ,None,None,РОССИЯ
1,АБГАРЯН ДИАНА АРКАДЬЕВНА,None,AR!!!!!!,АБГАРЯН,ДИАНА,АРКАДЬЕВНА,None,None,None,АРМЕНИЯ
2,АЙРОЯН ГЕВОРГ ЭДУАРД,None,AR!!!!!!,АЙРОЯН,ГЕВОРГ,ЭДУАРД,None,None,None,АРМЕНИЯ
3,АМРОЯН КАДЖИК ГЕВОРГОВИЧ,None,AR!!!!!!,АМРОЯН,КАДЖИК,ГЕВОРГОВИЧ,None,None,None,АРМЕНИЯ
4,АНИСЯН АНАИТ КОРЮНОВНА,None,AR!!!!!!,АНИСЯН,АНАИТ,КОРЮНОВНА,None,None,None,АРМЕНИЯ
...,...,...,...,...,...,...,...,...,...,...
563,ЧЕРШУКОВА АНЖЕЛИКА ВЛАДИМИРОВНА,None,АВ!!!!!!!!,ЧЕРШУКОВА,АНЖЕЛИКА,ВЛАДИМИРОВНА,None,None,None,УЗБЕКИСТАН
564,ШИТОВА ОЛЬГА ВАСИЛЬЕВНА,None,ET!!!!!!,ШИТОВА,ОЛЬГА,ВАСИЛЬЕВНА,None,None,None,УКРАИНА
565,ЭВАЙЗОВ АДИЛ ТАНРЫВЕРДИ ОГЛЫ,None,CO!!!!!!!,ЭВАЙЗОВ,АДИЛ,ТАНРЫВЕРДИ,ОГЛЫ,None,None,АЗЕРБАЙДЖАН
566,ЭЙВАЗОВА СААДАТ ЭЛЬЧИН КЫЗЫ,None,VIIМЮ!!!!!!,ЭЙВАЗОВА,СААДАТ,ЭЛЬЧИН,КЫЗЫ,None,None,РОССИЯ


Большинство наблюдений в тестовом датасете остались безродными космополитами: 

In [19]:
len(test_data.index) - len(doc_matched.index)

1932

Мы можем предположить их национальность по фамилии, хотя в случае, например, России и Украины такое предположение будет совсем хлипким.

Тем не менее, мы можем определить близость фамилии к фамилиям людей каждой из 6 национальностей в датасете и далее, использовать получившиеся 6 координат каждого наблюдения для оценки модели классификатора.

В данном случае, с учетом большого количества наблюдений, использовать косинусное расстояние затруднительно, поэтому ограничимся метрикой близости Левенштейна. Близость каждой фамилии к национальности мы будем измерять двумя способами: как среднее значение метрики Левенштейна для национальной группы фамилий и как максимальное значение для этой группы. Впоследствии мы используем ту метрику, которая лучше пройдет кроссвалидацию.

In [406]:
from difflib import SequenceMatcher

from sklearn.model_selection import train_test_split

from tqdm import tqdm

COUNTRIES = ('АРМЕНИЯ', 'УЗБЕКИСТАН', 'КИРГИЗИЯ', 'РОССИЯ', 'АЗЕРБАЙДЖАН',
       'УКРАИНА')

def similarity(word1, word2):
    return SequenceMatcher(None, word1, word2).ratio()

def country_matcher(to_match, method="max"):
    assert method in ("max", "mean"), f"Unknown method: {method}"
    meth = {"max" : np.max, "mean" : np.mean}[method]
    def _(country_grp):
        res = {"name0" : [], "similarity" : []}
        for name in tqdm(to_match):
            sims = [similarity(name, name2) for name2 in country_grp["name0"] if name != name2]
            res["name0"].append(name)
            res["similarity"].append(meth(sims))
        df = pd.DataFrame(res)
        return df
    return _

def filter_data(test_data, gender_matched, doc_matched, gender):
    b = test_data.copy().dropna().drop_duplicates()
    b = pd.merge(gender_matched, b[["name", "name0"]])
    return b.loc[(~b["name"].isin(doc_matched["name"])) & \
          (b["gender"]==gender), ["name0"]]

def make_match_data(fact_data, test_data, gender_matched, doc_matched, gender):
    assert gender in ("М", "Ж"), f"Unknown gender: {gender}"
    a = fact_data.loc[fact_data["gender"]==gender, ["name0", "country"]].copy().dropna().drop_duplicates()
    return a,filter_data(test_data, gender_matched, doc_matched, gender)

def make_train_test(fact_data, gender):
    a = fact_data.loc[fact_data["gender"]==gender, ["name0", "country"]].copy().dropna().drop_duplicates()
    return train_test_split(a, test_size=0.1)
    
def make_class_map(countries=COUNTRIES):
    return {country : num for num, country in enumerate(countries)}

def prep_train_data2(fact_data, gender, method="max"):
    assert gender in ("М", "Ж") or set(gender) == set(("М", "Ж")), \
        f"Unknown gender: {gender}"
    if isinstance(gender, str):
        gender= (gender,)
        
    inp = fact_data.loc[fact_data["gender"].isin(gender), ["name0", "country"]].copy().dropna().drop_duplicates()
    data = inp.groupby("country").apply(country_matcher(inp["name0"].unique(), method=method)).reset_index()
    a = data[["country", "name0", "similarity"]].copy()
    x_train = a.pivot(index="name0", columns="country", values="similarity").reset_index()
    factual = fact_data[["name0", "country"]].drop_duplicates()
    factual = factual.rename(columns={"country": "Y"})
    b = pd.merge(x_train, factual, how="left", on="name0").dropna()
    class_map = make_class_map()
    b = b.replace({'Y' : class_map})
    return b

def prep_train_data(fact_data, gender, method="max"):
    b = prep_train_data2(fact_data, gender, method)
    y_train = b[["Y"]]
    
    x_train = b.drop(columns=["name0","Y"])
    return x_train, y_train

def prep_test_data(test_data, fact_data, gender, method="max"):
    assert gender in ("М", "Ж") or set(gender) == set(("М", "Ж")), \
        f"Unknown gender: {gender}"
    if isinstance(gender, str):
        gender= (gender,)
    inp = test_data.loc[test_data["gender"].isin(gender), ["name0"]].copy().dropna().drop_duplicates()
    matches = fact_data.loc[fact_data["gender"].isin(gender), ["country", "name0"]].copy().dropna().drop_duplicates()
    data = matches.groupby("country").apply(country_matcher(inp["name0"].unique(), method=method)).reset_index()
    a = data[["name0", "similarity", "country"]].copy()
    x_train = a.pivot(index="name0", columns="country", values="similarity").reset_index()
    return x_train

    

In [358]:
x_all_mean, y_all_mean = prep_train_data(fact_data, ("М", "Ж"), method="mean")

100%|██████████| 5895/5895 [05:35<00:00, 17.60it/s]


In [359]:
x_all_mean

,АЗЕРБАЙДЖАН,АРМЕНИЯ,КИРГИЗИЯ,РОССИЯ,УЗБЕКИСТАН,УКРАИНА
0,0.252292,0.446424,0.256589,0.211945,0.222514,0.191153
1,0.356680,0.202826,0.351078,0.256737,0.344946,0.211899
2,0.389714,0.222226,0.403304,0.285845,0.378549,0.230623
3,0.248031,0.465927,0.264517,0.217423,0.220278,0.195915
4,0.261189,0.193962,0.277000,0.189799,0.215367,0.165926
...,...,...,...,...,...,...
6349,0.276106,0.158936,0.239187,0.239021,0.293336,0.196271
6350,0.199298,0.145608,0.183137,0.177942,0.214603,0.145741
6351,0.220250,0.166149,0.225580,0.244287,0.198514,0.236974
6352,0.180603,0.303201,0.215166,0.256164,0.213018,0.271006


In [171]:
x_all_max, y_all_max = prep_train_data(fact_data, ("М", "Ж"), method="max")

100%|██████████| 5895/5895 [05:23<00:00, 18.20it/s]


In [153]:
x_men_mean, y_men_mean = prep_train_data(fact_data, "М", method="mean")
x_men_max, y_men_max = prep_train_data(fact_data, "М", method="max")
x_women_mean, y_women_mean = prep_train_data(fact_data, "Ж", method="mean")
x_women_max, y_women_max = prep_train_data(fact_data, "Ж", method="max")

100%|██████████| 2246/2246 [00:46<00:00, 48.11it/s]


In [172]:
import xgboost as xgb
from sklearn.metrics import  classification_report

x_train = x_all_max
y_train = y_all_max

xgb_model = xgb.XGBClassifier(max_depth=5, 
                              learning_rate=0.1,
                              objective= 'multi:softprob',
                              n_jobs=-1).fit(x_train, y_train)
print (f"Model score: {xgb_model.score(x_train, y_train)}")

y_pred = xgb_model.predict(x_train)

print(classification_report(y_train, y_pred))
print (make_class_map())

Model score: 0.6666666666666666
              precision    recall  f1-score   support

           0       0.67      0.61      0.64       688
           1       0.87      0.95      0.91       547
           2       0.69      0.65      0.67      1184
           3       0.60      0.47      0.53      1426
           4       0.59      0.68      0.63       964
           5       0.67      0.77      0.72      1545

    accuracy                           0.67      6354
   macro avg       0.68      0.69      0.68      6354
weighted avg       0.66      0.67      0.66      6354

{'АЗЕРБАЙДЖАН': 0, 'АРМЕНИЯ': 1, 'КИРГИЗИЯ': 2, 'РОССИЯ': 3, 'УЗБЕКИСТАН': 4, 'УКРАИНА': 5}


Качество классификации для максимального значения подобия. Значения приведены для мужчин/женщин/всех наблюдений.


|     Страна   |      Точность     |     Полнота    |
|:------------ |:-----------------:|:--------------:|
| Азербайджан  |    0.69/0.60/0.67 | 0.53/0.65/0.61 |
| Армения      |    0.89/0.86/0.87 | 0.96/0.96/0.95 |
| Киргизия     |    0.69/0.72/0.69 | 0.59/0.74/0.65 |
| Россия       |    0.62/0.62/0.60 | 0.40/0.69/0.47 |
| Узбекистан   |    0.57/0.76/0.59 | 0.75/0.31/0.68 |
| Украина      |    0.71/0.73/0.67 | 0.85/0.69/0.77 |
  
Качество классификации для среднего значения подобия.

|     Страна   |      Точность     |    Полнота     | 
|:------------ |:-----------------:|:--------------:|
| Азербайджан  |    0.74/0.70/0.62 | 0.58/0.74/0.56 |  
| Армения      |    0.92/0.89/0.92 | 0.97/0.98/0.96 |  
| Киргизия     |    0.70/0.77/0.65 | 0.60/0.78/0.62 |  
| Россия       |    0.71/0.71/0.63 | 0.37/0.77/0.46 |  
| Узбекистан   |    0.54/0.87/0.55 | 0.82/0.38/0.49 |  
| Украина      |    0.73/0.80/0.68 | 0.83/0.80/0.79 |  

Если судить по F1 и его факторам, то наилучший результат классификатор показывает в случае Армении: при использовании максимальных значений метрики подобия он улавливает 96% реальных граждан этой страны в фактическом датасете (и для мужчин и для женщин), хотя точность классификации несколько, но не критично, ниже. Наихудший результат у Узбекистана - всего 31% реальных граждан признаны таковыми, хотя узнает не-граждан Узбекистана классификатор относительно неплохо (точность 0.76). 

Стоит отметить, что мы не используем стандартное разделение датасета на тренировочный и тестовый и не валидируем результаты оценки модели на тестовой выборке, поскольку нашей целью является извлечение максимума полезной информации из "фактических" данных и использование этой информации для классификации "тестовых", каковое возможно только в режиме "без учителя". Поэтому нас интересует только качество классификации на тренировочных данных. 

Другими словами, для того, чтобы валидировать оценку модели на тестовой подвыборке из фактических данных нам нужно было бы цензурировать фактические данные. Для классификации тестового датасета мы должны будем использовать оценку модели, полученную для полного набора фактических данных (во избежание потери информации), что сделает валидацию на подвыборке бессмысленной.

## Преобразование документов

Качество классификатора, полученного на основе фамилий, следует признать относительно низким. Попытаться повысить его мы можем используя информацию, содержащуюся в масках номеров документов. Для этого мы извлечем из этих масок информацию в виде новых переменных:

* дамми "страновых" комбинаций букв - сгруппируем уникальные последовательности букв в масках по странам и присвоим значение 1 соответствующей страновой дамми, если последовательность букв в маске документа наблюдения входит в группу последовательностей этой страны;

* переменная длины строки маски;

* переменная длины последовательности букв в маске;

* дамми на присутствие последовательности букв в начале строки маски;

* дамми на присутствие последовательности букв в любом месте строки маски, кроме ее начала. 

In [418]:
country = "АЗЕРБАЙДЖАН"
docs_by_cntry = fact_data[["country", "document"]].drop_duplicates()
doc_classes = {"АЗЕРБАЙДЖАН" : ["^(P|C|AT)[!]{7}$", 
                                "^(?:AZII|АИ|I{2,3}БА|VИК)[!]{6}$", 
                                "^[!]{9}$",
                               "^PPAZE(?:Р|С|)[!]{6,7}$",
                               "^PCAZEС[!]{8}$"],
              "РОССИЯ" : []}

def letters_len(df):
    a = df.copy()
    b = a["document"].str.extract("(?P<letters>[^!]+)")
    b.loc[:, "len"] = b["letters"].str.len()
    b = b.fillna(0)
    return b["len"]

def total_len(df):
    a = df.copy()
    b = a["document"].str.len()
    b = b.fillna(0)
    return b

def letters_country(df, country):
    a = df[df["country"]==country][["country", "document"]].copy().drop_duplicates()
    b = a["document"].str.extract("^(?P<letters>[^!]+)").dropna()
    return set("".join(b["letters"]))

def letter_combos_country(df, country):
    a = df[df["country"]==country][["document"]].copy().drop_duplicates()
    b = a["document"].str.extract("(?P<letters>[^!]+)").dropna()
    return b["letters"].unique()

def letters_begin(df):
    a = df[["document"]].copy()
    b = a["document"].str.extract("^(?P<letters>[^!]+)")
    b.loc[:, "dummy"] = 0
    b.loc[~b["letters"].isna(), "dummy"] = 1
    return b["dummy"]

def letters_mid(df):
    a = df[["document"]].copy()
    b = b = a["document"].str.extract("^[!]+(?P<letters>[^!]+)[!]+")
    b.loc[:, "dummy"] = 0
    b.loc[~b["letters"].isna(), "dummy"] = 1
    return b["dummy"]

def make_combos(df):
    a = df.copy()
    countries = make_class_map()
    return {country : letter_combos_country(a, country) for country in countries.keys()}

def make_country_combo_dummies(df, combos=None):
    a = df.copy()
    countries = make_class_map()
    if combos is None:
        combos = make_combos(a)
        
    b = a["document"].str.extract("(?P<combo>[^!]+)")
    a.loc[:, "combo"] = b["combo"]
    for country, num in countries.items():
        a.loc[:, f"c{num}_combo"] = 0
        idx = (a["combo"].isin(combos[country]))
        a.loc[idx, f"c{num}_combo"] = 1
    a = a.drop(columns="combo")
    return a

def letters_begin_dummy(df):
    a = df.copy()
    a.loc[:, "begin"] = letters_begin(a)
    return a

def letters_mid_dummy(df):
    a = df.copy()
    a.loc[:, "mid"] = letters_mid(a)
    return a

fact_df = letters_mid_dummy(letters_begin_dummy(make_country_combos(fact_data)))
fact_df.loc[:, "letters_len"] = letters_len(fact_df)
fact_df.loc[:, "total_len"] = total_len(fact_df)
fact_df

,name,gender,document,country,name0,name1,name2,name3,name4,name5,c0_combo,c1_combo,c2_combo,c3_combo,c4_combo,c5_combo,begin,mid,letters_len,total_len
0,АБАДЖЯН МКРТИЧ МКРТЫЧЕВИЧ,М,AP!!!!!!,АРМЕНИЯ,АБАДЖЯН,МКРТИЧ,МКРТЫЧЕВИЧ,None,None,None,1,0,0,0,0,0,1,0,2.0,8
1,АБАДУЛЛАЕВ РАШИДБАЙ КАНДАХАРОВИЧ,М,AB!!!!!!!,УЗБЕКИСТАН,АБАДУЛЛАЕВ,РАШИДБАЙ,КАНДАХАРОВИЧ,None,None,None,1,1,0,0,0,1,1,0,2.0,9
2,АБАЕВ ЙУЛДОШ ЮЛДОШОВИЧ,М,СР!!!!!!!,УЗБЕКИСТАН,АБАЕВ,ЙУЛДОШ,ЮЛДОШОВИЧ,None,None,None,0,1,0,0,0,1,1,0,2.0,9
3,АБАЗЯН ГЕВОРГ ЕНОКОВИЧ,М,AP!!!!!!,АРМЕНИЯ,АБАЗЯН,ГЕВОРГ,ЕНОКОВИЧ,None,None,None,1,0,0,0,0,0,1,0,2.0,8
4,АБАЙ УУЛУ ДАНИЯРБЕК МИХАЙЛОВИЧ,М,,КИРГИЗИЯ,АБАЙ,УУЛУ,ДАНИЯРБЕК,МИХАЙЛОВИЧ,None,None,0,0,0,0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,ЯХШИЕВ ТУРГУН ЮЛДАШЕВИЧ,М,АВ!!!!!!!,УЗБЕКИСТАН,ЯХШИЕВ,ТУРГУН,ЮЛДАШЕВИЧ,None,None,None,1,1,0,1,0,1,1,0,2.0,9
9496,ЯХЬЯЙЕВ БАЙРАМ ВИЛАДИ ОГЛЫ,М,,АЗЕРБАЙДЖАН,ЯХЬЯЙЕВ,БАЙРАМ,ВИЛАДИ,ОГЛЫ,None,None,0,0,0,0,0,0,0,0,0.0,0
9497,ЯЦЕВСКАЯ ТАТЬЯНА НИКОЛАЕВНА,Ж,МН!!!!!!,УКРАИНА,ЯЦЕВСКАЯ,ТАТЬЯНА,НИКОЛАЕВНА,None,None,None,0,0,0,0,0,1,1,0,2.0,8
9498,ЯЦЕНКО АЛЕКСАНДР ИОСИФОВИЧ,М,МН!!!!!!,УКРАИНА,ЯЦЕНКО,АЛЕКСАНДР,ИОСИФОВИЧ,None,None,None,0,0,0,0,0,1,1,0,2.0,8


Восстановим тренировочные данные с фамилиями. Поскольку максимальные значения давали несколько лучшие значения F1, то остановимся на них и не будем разделять наблюдения по полу:

In [380]:
train_data = prep_train_data2(fact_data, gender=("М", "Ж"))

100%|██████████| 5895/5895 [05:14<00:00, 18.77it/s]


Объединим все переменные:

In [394]:
a = fact_df[["name0", 'c0_combo', 'c1_combo', 'c2_combo',
       'c3_combo', 'c4_combo', 'c5_combo', 'begin', 'mid', 'letters_len',
       'total_len']].copy()
train_all = pd.merge(a, train_data, on="name0")
#train_all = train_all[[name for name in make_class_map().keys()] + ['c0_combo', 'c1_combo', 'c2_combo',
#       'c3_combo', 'c4_combo', 'c5_combo', 'begin', 'mid', 'letters_len',
#       'total_len', "Y", "name0"]].sort_values("name0")
b = train_all.drop_duplicates()
x_train = b[[col for col in b.columns if col not in ("name0", "Y")] ]
y_train = b[["Y"]]

xgb_model = xgb.XGBClassifier(max_depth=5, 
                              learning_rate=0.1,
                              objective= 'multi:softprob',
                              n_jobs=-1).fit(x_train, y_train)
print (f"Model score: {xgb_model.score(x_train, y_train)}")

y_pred = xgb_model.predict(x_train)

print(classification_report(y_train, y_pred))
print(make_class_map())

Model score: 0.72567057228624
              precision    recall  f1-score   support

           0       0.86      0.96      0.91      1215
           1       0.70      0.72      0.71      1742
           2       0.77      0.73      0.75      1857
           3       0.65      0.62      0.64      2064
           4       0.62      0.75      0.68      1593
           5       0.81      0.68      0.74      1856

    accuracy                           0.73     10327
   macro avg       0.74      0.74      0.74     10327
weighted avg       0.73      0.73      0.73     10327

{'АРМЕНИЯ': 0, 'УЗБЕКИСТАН': 1, 'КИРГИЗИЯ': 2, 'РОССИЯ': 3, 'АЗЕРБАЙДЖАН': 4, 'УКРАИНА': 5}


Сравнение качества классификации: максимальные значения метрики подобия, без разделения по полу, без доп. переменных / с доп.переменными

|     Страна   |   Точность   |  Полнота  |
|:------------ |:------------:|:---------:|
| Азербайджан  |    0.67/0.62 | 0.61/0.75 |
| Армения      |    0.87/0.86 | 0.95/0.96 |
| Киргизия     |    0.69/0.77 | 0.65/0.73 |
| Россия       |    0.60/0.65 | 0.47/0.62 |
| Узбекистан   |    0.59/0.70 | 0.68/0.72 |
| Украина      |    0.67/0.81 | 0.77/0.68 |

Добавление переменных несколько улучшило качество классификации и мы можем перейти к присвоению наблюдениям предположительной национальности.

Составим таблицу эндогенных переменных на основе тестовых данных:

In [420]:

test_df = letters_mid_dummy(letters_begin_dummy(make_country_combo_dummies(test_data, make_combos(fact_data) )))
test_df.loc[:, "letters_len"] = letters_len(test_df)
test_df.loc[:, "total_len"] = total_len(test_df)
test_df


,name,gender,document,country,name0,name1,name2,name3,name4,name5,c0_combo,c1_combo,c2_combo,c3_combo,c4_combo,c5_combo,begin,mid,letters_len,total_len
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,None,С!!!!!!!,None,АБАСОВ,РАШАД,РАДЖАБ,ОГЛЫ,None,None,1,1,0,0,1,1,1,0,1.0,8
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,None,!!!!!!!!!!,None,АБАСОВА,УЛЬЯНА,ИГОРЕВНА,None,None,None,0,0,0,0,0,0,0,0,0.0,10
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,None,!!!!!!!!!!,None,АББАСОВ,ДЖЕЙХУН,ДАВУД,ОГЛЫ,None,None,0,0,0,0,0,0,0,0,0.0,10
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,None,З!!!!!!!,None,АББАСОВ,ДУНЯМИН,МАХМУД,ОГЛЫ,None,None,0,0,0,0,0,0,1,0,1.0,8
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,None,С!!!!!!!,None,АББАСОВ,ЕЛЬНУР,АРИЗ,ОГЛЫ,None,None,1,1,0,0,1,1,1,0,1.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,ЯКУШЕВА АНАСТАСИЯ АЙДАРОВНА,None,!!!!!!!!!!,None,ЯКУШЕВА,АНАСТАСИЯ,АЙДАРОВНА,None,None,None,0,0,0,0,0,0,0,0,0.0,10
2496,ЯРОСЛАВЦЕВ ДАНИЛ ИВАНОВИЧ,None,VАР!!!!!!,None,ЯРОСЛАВЦЕВ,ДАНИЛ,ИВАНОВИЧ,None,None,None,0,0,0,1,0,0,1,0,3.0,9
2497,ЯСИК АЛЕКСАНДР ВАСИЛЬЕВИЧ,None,!!!!!!!!!!,None,ЯСИК,АЛЕКСАНДР,ВАСИЛЬЕВИЧ,None,None,None,0,0,0,0,0,0,0,0,0.0,10
2498,ЯХЬЯЕВА САНИЯ ОРУДЖ КЫЗЫ,None,P!!!!!!!,None,ЯХЬЯЕВА,САНИЯ,ОРУДЖ,КЫЗЫ,None,None,1,0,0,1,1,0,1,0,1.0,8


In [ ]:
data = pd.merge(full_match, test_data[["name", "name0"]].drop_duplicates(), how="left")
test_prep = prep_test_data(data, fact_data, ("М", "Ж"), method="max")


In [445]:
a = test_df[["name0", 'c0_combo', 'c1_combo', 'c2_combo',
       'c3_combo', 'c4_combo', 'c5_combo', 'begin', 'mid', 'letters_len',
       'total_len']]
x_test = pd.merge(a, test_prep, on="name0")
x_test_name0 = x_test["name0"]
x_test = x_test.drop(columns="name0")
test_pred = xgb_model.predict(x_test)
result = pd.DataFrame({"name0" : x_test_name0,
                      "country" : test_pred})
country_map = {v:k for k,v in make_class_map().items()}
result = result.replace({"country" : country_map})
result.loc[:, "name"] = test_df["name"]
result.loc[:, "document"] = test_df["document"]
b = result[["name",  "document", "country"]].copy()
final = pd.merge(b, full_match, on="name")
final = final[["name", "gender", "document", "country"]]
final

Запишем результат в исходную таблицу.